<a href="https://colab.research.google.com/github/Norod/hebrew-gpt_neo/blob/main/hebrew_gpt_neo_with_Javascript_interfaceipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/

######################################################################
#                                                                    #
model_name = "Norod78/hebrew-gpt_neo-xl"
#model_name ="Norod78/hebrew-gpt_neo-xl-poetry"
#                                                                    #
#model_name = "Norod78/hebrew-gpt_neo-small"
#model_name = "Norod78/hebrew_poetry-gpt_neo-small"
#model_name = "Norod78/hebrew-project_ben_yehuda-gpt_neo-small"
#model_name = "Norod78/hebrew_stories-gpt_neo-small"
#                                                                    #
#model_name = "Norod78/hebrew-gpt_neo-tiny"
#model_name = "Norod78/hebrew_poetry-gpt_neo-tiny"
#model_name = "Norod78/hebrew-bad_wiki-gpt_neo-tiny"
#                                                                    #
######################################################################

seed = None
stop_token = "<|endoftext|>"
new_lines = "\n\n\n"

tokenizer_name = model_name

/content


In [ ]:
%cd /content/

!pip install -q tokenizers==0.10.3 transformers==4.8.0

/content


In [ ]:
%cd /content/

import argparse
import re
import os
import requests

import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(tokenizer_name, pad_token_id=tokenizer.eos_token_id)

np.random.seed(seed)
random_seed = np.random.randint(10000,size=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = 0 if torch.cuda.is_available()==False else torch.cuda.device_count()

torch.manual_seed(random_seed)
if n_gpu > 0:
    torch.cuda.manual_seed_all(random_seed)

model.to(device)

/content


GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0, inplace=False)
    (h): ModuleList(
      (0): GPTNeoBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0, inplace=False)
            (resid_dropout): Dropout(p=0, inplace=False)
            (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=2048, out_features=8192, bias=True)
          (c_proj): Linear(in_fea

In [ ]:
!git clone https://github.com/gpt2ent/gpt2colab-js.git
%cd gpt2colab-js

fatal: destination path 'gpt2colab-js' already exists and is not an empty directory.
/content/gpt2colab-js


In [ ]:
generate_count = 0
spinner_speed = "1200ms"

import google.colab.output

def overlap(a, b):
    return max(i for i in range(len(b)+1) if a.endswith(b[:i]))


def ai_generate(input_text, top_p, top_k, max_size):
    global generate_count

    top_p = float(top_p)
    top_k = int(top_k)
    length = int(max_size)

    if len(input_text) == 0:
        input_text = " "

    encoded_prompt = tokenizer.encode(
    input_text, add_special_tokens=False, return_tensors="pt")

    encoded_prompt = encoded_prompt.to(device)

    if encoded_prompt.size()[-1] == 0:
        input_ids = None
    else:
        input_ids = encoded_prompt
    
    output_sequences = model.generate(
    input_ids=input_ids,
    max_length=length + len(encoded_prompt[0]),
    top_k=top_k, 
    top_p=top_p, 
    do_sample=True,
    num_return_sequences=1)

    # Remove the batch dimension when returning multiple sequences
    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

    generated_sequences = []

    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):        
        generated_sequence = generated_sequence.tolist()

        # Decode text
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

        # Remove all text after the stop token
        text = text[: text.find(stop_token) if stop_token else None]

        # Remove all text after 3 newlines
        text = text[: text.find(new_lines) if new_lines else None]

        # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
        total_sequence = (
            input_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
        )

        generated_sequences.append(total_sequence)
    
    parsed_text = total_sequence.replace("<|startoftext|>", "").replace("\r","").replace("\n\n", "\n")
    if len(parsed_text) == 0:
        parsed_text = "שגיאה"
    else:
      j = overlap(input_text, parsed_text)
      if j > 0 :      
        parsed_text = parsed_text[j:]

    generate_count += 1
    return parsed_text

#register callback for Javascript
google.colab.output.register_callback('ai_generate', ai_generate)

print('Done')

Done


In [ ]:
%cd /content/gpt2colab-js/
ai_generate("ברוכים הבאים", 0.92, 50, 10)

/content/gpt2colab-js


' לעולם המיחזור. אם אתם מתעניינים בנו'

In [ ]:
from IPython.display import HTML

#spinner from https://codepen.io/vovchisko/pen/vROoYQ
spinner_css = """
<style>
@keyframes c-inline-spinner-kf {
  0% {
    transform: rotate(0deg);
  }
  100% {
    transform: rotate(360deg);
  }
}

.c-inline-spinner,
.c-inline-spinner:before {
  display: inline-block;
  width: 11px;
  height: 11px;
  transform-origin: 50%;
  border: 2px solid transparent;
  border-color: #74a8d0 #74a8d0 transparent transparent;
  border-radius: 50%;
  content: "";
  animation: linear c-inline-spinner-kf """+spinner_speed+""" infinite;
  position: relative;
  vertical-align: inherit;
  line-height: inherit;
}
.c-inline-spinner {
  top: 3px;
  margin: 0 3px;
}
.c-inline-spinner:before {
  border-color: #74a8d0 #74a8d0 transparent transparent;
  position: absolute;
  left: -2px;
  top: -2px;
  border-style: solid;
}
</style>
"""

input_form = """
<link rel="stylesheet" href="https://unpkg.com/purecss@1.0.1/build/pure-min.css" integrity="sha384-oAOxQR6DkCoMliIh8yFnu25d7Eq/PHS21PClpwjOTeU2jRSq11vu66rf90/cZr47" crossorigin="anonymous">

<div style="background-color:white; border:solid #ccc; width:800px; padding:20px; color: black;">
<p>You have currently loaded </p>
<p> %s model</p>
<textarea dir="rtl" id="main_textarea" cols="40" rows="10" style="font-family: 'Liberation Serif', 'DejaVu Serif', Georgia, 'Times New Roman', Times, serif; font-size: 13pt; padding:10px;"></textarea><br>
<div class="pure-form pure-form-aligned">
    <div class="pure-control-group">
      <label for="top_p">top_p:</label>
      <input type="number" min="0.00" max="1.00" step="0.5" id="top_p" value="0.95" style="background-color: white;">
    </div>
    <div class="pure-control-group">
        <label for="top_k">top_k:</label>
        <input type="number" min="0" max="9999" id="top_k" value="50" style="background-color: white;">
    </div>
    <div class="pure-control-group">
        <label for="length">Generate how much:</label>
        <input type="number" id="length" min="1" max="2043" value="48" style="background-color: white;">
    </div>
    <div style="width: 300px; display: block; margin-left: auto !important; margin-right: auto !important;">
        <p><button class="pure-button pure-button-primary" style="font-size: 125%%;" onclick="generate()">המשך</button>
        <span class="c-inline-spinner" style="visibility: hidden;" id="spinner"></span></p>
    </div>
</div>
</div>
""" % model_name

javascript = """
<script type="text/Javascript">
    function desanitize(text) {
        return text.slice(1,-1).replace(/\\\\n/g, "\\n").replace(/\\\\'/g, "'");
    };

    function add_text(text) {
        var deftext = document.getElementById('main_textarea').value;
        document.getElementById('main_textarea').value = deftext + text;
    };

    function generate(){
        var prefix = document.getElementById('main_textarea').value;
        var top_p = document.getElementById('top_p').value;
        var top_k = document.getElementById('top_k').value;
        var length = document.getElementById('length').value;
        
        var kernel = google.colab.kernel;
        var resultPromise = kernel.invokeFunction("ai_generate", [prefix,top_p,top_k,length]); // developer, look here
        resultPromise.then(
            function(value) {
              add_text(desanitize(value.data["text/plain"]));
              document.getElementById('spinner').style = "visibility: hidden;";
        });
        document.getElementById('spinner').style = "visibility: visible;";
    };
</script>
"""

HTML(spinner_css + input_form + javascript)